![MAA MathFest 2016](http://www.maa.org/sites/default/files/images/mathfest/2016_mathfest_banner_with_dates.gif "MAA MathFest 2016")
Syntactically Informed Text Compression with Recurrent Neural Networks
======
## David Cox, Dr. J. Maurice Rojas
## Texas A&M University


# About Me

* Texas A&M University class of 2016
* Computer science, mathematics
* Malware analysis at Palo Alto Networks

![Texas A&M University](http://brandguide.tamu.edu/downloads/logos/TAMU-logos-rgb/TAM-
Wordmark/TAM-Wordmark.png "Texas A&M University")


* This is my first time speaking at a conference.
 * This is also my first time attending a math conference.
 * (bear with me!)
* My slides were generated using IPython Notebook.
 * If you like the format, now you know.
 * If you hate it... sorry.

# Project Overview

We explored using recurrent neural networks to construct language models for text compression.

![Dr. Mahoney](https://cs.fit.edu/~mmahoney/matt.jpg "Dr. Mahoney")
We built upon work done by Dr. Mahoney at Florida Tech.

* He used a 4 x 1,000,000 x 1 fully connected neural network optimized for single-pass training to produce bit-level probability estimates for text sequences.

In his publication, Dr. Mahoney notes that this model does not take advantage of syntactic or semantic relationships present in natural language. 

* He cites the ability to exploit these relationships as the motivation behind using neural networks in the first place. 

We utilized Google's state-of-the-art natural language parser to obtain highly accurate syntactic information in the form of part of speech tags. 
![Parsey McParseFace](https://2.bp.blogspot.com/-fqtmVS97tOs/VzTEAI9BQ8I/AAAAAAAAA_U/xPj0Av64sGseS0rF4Z1BbhmS77J-HuEvwCLcB/s1600/image04.gif "Parsey McParseface")

* Google's parser is named Parsey McParseface, taking after the internet meme "Boaty McBoatface"

![Boaty](https://www.dailywire.com/sites/default/files/maxresdefault_0.jpg "Boaty")

We also used a recurrent neural network architecture better suited to modeling sequences of data, such as text.

![Recurrent Neural Network](https://d3kbpzbmcynnmx.cloudfront.net/wp-content/uploads/2015/09/rnn.jpg "Recurrent Neural Network")

### We found that the addition of syntactic information improved the accuracy of our model by an average of 5%.

![Accuracy](./report/images/tags_vs_no-tags.png "Accuracy")

### Bottom line

* Providing a neural network with more, valid information results in a more accurate model.
* In our case, the model can then be used for text compression.

# Background

## Computer science disclaimer

Data compression and neural networks have roots in both mathematics and computer science. We've tried to make this talk as accessible possible, but please ask for further explanation at the end of the talk if something isn't clear. 

# Data Compression

* Data compression is the process of encoding a message with the goal of reducing the number of symbols required to represent the message. 

## Compression operates in two steps
* Modeling
 * Modeling is the process of generating an estimate for the expected probability of symbols in the input.
 * Accurate probability estimates result in efficient compression. 
* Coding
 * Coding is the process of translating probability estimates to a sequence of symbols. 

### Modern coding methods are theoretically optimal, while modeling is considered an "unsolvable problem". 

### Consider an optimal model

* An optimal model could predict the next symbol in an arbitrary sequence with 100% accuracy.
* A compression scheme based upon this model would be able to recursively compress its own output to zero bytes.
 * This would violate Kolmogorov's principle of string complexity. 

## The current best approach to improving compression is to construct domain-specific models and interface them with a near-optimal coding method.

## Arithmetic coding is a popular, near-optimal coding method that operates by representing a sequence of probabilities as a fractional number in the interval [0,1).

### Consider the following example
* Let $M$ be a message composed of symbols: [C, O, D, E, !]
* Let $S$ be an alphabet containing the symbols {A, B, C, D, E, O, !}
* The following table contains an arbitrary, fixed probability model for the symbols in $S$.

| Symbol        | Probability   | Range      |
| :-----------: |:-------------:| :-----     |
| A             | 0.3           | [0, 0.3)   |
| B             | 0.2           | [0.3, 0.5) |
| C             | 0.2           | [0.5, 0.7) |
| D             | 0.1           | [0.7, 0.8) |
| E             | 0.1           | [0.8, 0.9) |
| O             | 0.05          | [0.9, 0.95)|
| !             | 0.05          | [0.95 ,1)  | 


### We encode $M$ by reducing the range of our subinterval from its initial range of [0,1) for each symbol as shown:

| Step          | After         | Range               |
| :-----------: |:-------------:| :------------------:|
| 0             | $\emptyset$   | [0, 1)              |
| 1             | C             | [0.5, 0.7)          |
| 2             | O             | [0.680, 0.690)      |
| 3             | D             | [0.678, 0.688)      |
| 4             | E             | [0.6878, 0.6879)    |
| 5             | !             | [0.687895, 0.68790) |
| Done          | CODE!         | 0.687895            |  

* In this example the output ended up requiring more symbols than the input, which is typically not the case. 
* This was caused by using an arbitrary probability model. Our unfortunate example highlights the need for accurate language models.

### We just used a static model to perform arithmetic coding, but models can also be dynamic. 

* Dynamic models allow for probability estimates to be updated based on the symbols or characters that have been processed. 
* Neural networks can be used as dynamic models.

# Recurrent Neural Networks

### Recurrent neural networks are a class of neural networks well suited for modeling sequential data such as audio or text. 

* RNN's excel in these domains due to memory provided by a recurrence in their hidden layers.
* This memory allows for the representation of contextual dependencies over arbitrary time intervals. 
* We can represent the hidden state of a RNN with a simple set of recurrence equations:
$$
\begin{aligned}
net_j(t) &= \sum_i^nx_i(t)v_{ji} + \sum_h^m y_h(t-1)u_{jh} + \theta_j\\
y_j(t) &= f\left(net_j(t)\right)\\
\end{aligned}
$$

where $y_j(t)$ is the output of the hidden state at time $t$ and $y_h(t-1)$ is the hidden state from the previous time interval. 

* Vectors $\mathbf{x}, \mathbf{V}, \mathbf{U}$ and $\mathbf{W}$ are weights:
 * input
 * input-hidden
 * hidden-hidden
 * hidden-out
* Each layer is assigned an index variable
 * $k$ for output nodes
 * $j,h$ for hidden nodes
 * $i$ for input nodes
* The functions $f$ and $g$ are differentiable, nonlinear activation functions such as the sigmoid or hyperbolic tangent function. 
* $\theta_j$ is a bias.
* The output state $y_k(t)$ can be computed as:

$$
\begin{aligned}
net_k(t) &= \sum_j^m y_j(t)w_{kj} + \theta_k\\
y_k(t) &= g\left(net_k(t)\right)\\
\end{aligned}
$$

All together, we see that a single forward pass through the network can be calculated with the following recurrence:

$$
\begin{aligned}
y_k(t) &= g\left( \sum_j^m \left( f\left(  \sum_i^nx_i(t)v_{ji} + \sum_h^m y_h(t-1)u_{jh} + \theta_j  \right) \right) w_{kj} + \theta_k \right)\\
\end{aligned}
$$

# Backpropagation Through Time

### To allow for learning over arbitrary intervals, error values must be backpropagated through time. 

We use the cross entropy error function in our model, defined as

$$
\begin{aligned}
C &= \frac{1}{2} \sum_p^n H(d,y)\\
\end{aligned}
$$

for the $p$th sample in the training set of length $n$ and the cross entropy function $H(p,q)$:

$$
\begin{aligned}
H(p,q) &= - \sum_x p(x) \log q(x)\\
\end{aligned}
$$

All together, our error function is:

$$
\begin{aligned}
C &= \frac{1}{2} \sum_p^n \left( - \sum_k^m d_{pk} \log (y_{pk}) \right)\\
\end{aligned}
$$

for $d$, the desired output of $m$ output nodes.

Weight updates are proportional to the negative cost gradient with respect to the weight that is being updated, scaled by the learning rate, $\eta$:

$$
\begin{aligned}
\Delta w &= - \eta \frac{\partial C}{\partial w}\\
\end{aligned}
$$

We can then compute the output error, $\delta_{pk}$ and hidden error, $\delta_{pj}$, which can be backpropagated through time to obtain the error of the hidden layer at the previous time interval.

Indices $h$ and $j$ are for nodes sending and receiving the activation, respectively.

$$
\begin{aligned}
\delta_{pk} &= \frac{\partial C}{\partial y_{pk}} \frac{\partial y_{pk}}{\partial net_{pk}}\\
\delta_{pj} &= - \left( \sum_k^m \frac{\partial C}{\partial y_{pk}} \frac{\partial y_{pk}}{\partial net_{pk}} \frac{\partial net_{pk}}{y_{pj}}\right) \frac{\partial y_{pj}}{\partial net_{pj}}\\
&= \sum_k^m \delta_{pk}w_{kj}f'(y_{pj}) \\
\delta_{pj}(t-1) &= \sum_h^m \delta_{ph}(t)u_{hj}f' \left( y_{pj}(t-1) \right)\\
\end{aligned}
$$

# Gated Recurrent Units


### While recurrent neural networks are more suited to this problem than the feedforward networks used by Dr. Mahony, they come with a significant drawback.

### When backpropagation is done over many timer intervals, error gradients can either vanish or "explode". This prevents the model from learning and reduces accuracy. 

### A popular solution to this problem is the use of Gated Recurrent Units (GRUs).

* Networks of gated recurrent units allow for modeling of multiple long-term dependencies at arbitrary time scales. 
 * They achieve this by allowing gates to reset themselves.
* A single GRU consists of a hidden state along with reset and update gates. 
 * When the reset gate, $r_j$ is closed $(r_j = 0)$, the value of the GRU's previous hidden state is ignored.
 * This resets the unit.
 
The value of the reset gate is computed as:

$$ 
\begin{aligned}
r_j = \sigma \left( v_{jr} x_i + u_{jr} y_{h}(t-1)\right)\\
\end{aligned}
$$

for the sigmoid activation function $\sigma(t) = \left(1+e^t\right)^{-1}$, the unit's input and previous hidden state, $x_i$ and $y_{h}(t-1)$, respectively. The weight matrices $\mathbf{V}$ and $\mathbf{U}$ follow from our previous equations.

The update gate $z_j$ is similar:

\begin{align*}
z_j = \sigma \left( v_{jz} x_i + u_{jz} y_{h}(t-1)\right)\\
\end{align*}


$\widetilde{y}$ is pronounced "y wiggle"

The new hidden state, $\widetilde{y}_{j}(t)$ is:

\begin{align*}
\widetilde{y}_{j}(t) = \tanh \left( v_{j} x_{j} + u_j\left(r_j y_{h}(t-1)\right) \right) \\
\end{align*}

*Note the role of the reset gate in the calculation of the new hidden state.*


Finally, the unit's activation function, $y_{j}(t)$ can be calculated as a linear interpolation between the previous and current states:

\begin{align*}
y_{j}(t) = z_j y_{h}(t-1) + (1-z_j)\widetilde{y}_{j}(t) \\
\end{align*}

The output of a single forward pass in a single layer GRU network can be represented using notation from the previous simple recurrent model:

\begin{align*}
net_k(t) &= \sum_j^m y_j(t)w_{kj} + \theta_k\\
  &= \sum_j^m \left( z_j y_{h}(t-1) + (1-z_j)\widetilde{y}_{j}(t)\right) w_{kj} + \theta_k\\
y_k(t) &= g\left(net_k(t)\right)\\
\end{align*}


# Model Architecture

### Our background information followed a series of problems.
* The need for an effective language model was addressed.
 * An ideal neural network architecture was discussed,
   * and an improved hidden unit was selected.
   
This part of the talk will address the issue of improving upon a vanilla GRU network architecture that operates solely on character sequences. The improvements discussed occur at a higher level of abstraction than the gate level architectures previously described, as we are seeking to build a practical model rather than propose a new recurrent unit architecture.

### Jumping right in: Implementation with Keras

In [ ]:
char_encoder = Sequential(name='char_encoder')
pos_encoder  = Sequential(name='pos_encoder')
char_encoder.add(
        GRU(
            output_dim=256,
            return_sequences=True,
            input_shape=(maxlen, len(chars)),
            consume_less='gpu',
            )
        )
char_encoder.add(Dropout(0.1))
pos_encoder.add(
        GRU(
            output_dim=49,
            return_sequences=True,
            input_shape=(maxlen, len(tags)),
            consume_less='gpu',
            )
        )
pos_encoder.add(Dropout(0.1))
decoder = Sequential(name='decoder')
decoder.add(Merge([char_encoder, pos_encoder], mode='concat'))
decoder.add(
        GRU(
            output_dim=305,
            return_sequences=False,
            consume_less='gpu',
            )
        )
decoder.add(Dense(len(chars), activation='relu'))
decoder.add(Dense(len(chars), activation='softmax'))
decoder.compile(loss='categorical_crossentropy', optimizer='rmsprop')

![Model Architecture](./report/model_architecture.png "Model Architecture")


## Notation used:

| Layer                                    | Description           | 
| :----------------------------------------|:----------------------| 
| $\mathbf{x}^{\langle c \rangle}$      | Character input layer | 
| $\mathbf{x}^{\langle p \rangle}$      | POS input layer       | 
| $\mathbf{y}^{\langle c \rangle}$      | GRU layer (character) | 
| $\mathbf{y}^{\langle p \rangle}$      | GRU layer (POS)       | 
| $\mathbf{\Xi}^{\langle c p \rangle}$  | Dropout layer         |  
| $\mathbf{\Psi}^{\langle c,p \rangle}$ | Merge layer           | 
| $\mathbf{y}^{\langle \Psi \rangle}$   | GRU layer (merged)    | 
| $\mathbf{y}^{\langle D1 \rangle}$     | Dense layer: RELU     | 
| $\mathbf{y}^{\langle D1 \rangle}$     | Dense layer: Softmax  | 
| $\mathbf{y}^{\langle out \rangle}$    | Network output        | 

The character input layer, $\mathbf{x}^{\langle c \rangle}(t)$, is a $40 \times 256$ one-hot representation of forty character sequences. This layer is paralleled by a second input layer containing part of speech information obtained from SyntaxNet. 

The part of speech tag (POS) input layer, $\mathbf{x}^{\langle p \rangle}(t)$ is a $40 \times 49$ one-hot representation of part of speech tag sequences, each of which correspond to the character at the same respective index in the other input layer.

One-hot encoding is a sparse way to represent information in which a single bit in an array is "hot" (1), with the rest being low (0).
![one-hot](https://cd8ba0b44a15c10065fd-24461f391e20b7336331d5789078af53.ssl.cf1.rackcdn.com/graphlab.vanillaforums.com/editor/v3/b3s53janx6l2.png "one-hot")

GRU layers $\mathbf{y}^{\langle c \rangle}(t)$ and $\mathbf{y}^{\langle p \rangle}(t)$ are also parallel. We will use the notation $\mathbf{y}^{\langle c|p \rangle}(t)$ when discussing separate but identical operations to both layers. Our implementation utilizes the hard (linearly approximated) sigmoid function in place of the standard logistic sigmoid as the GRU's inner activation function in order to reduce computational requirements. The outer activation, $g$ is the hyperbolic tangent function applied element-wise for each node in the layer. 

A forward pass through $\mathbf{y}^{\langle c \rangle}(t)$ and $\mathbf{y}^{\langle p \rangle}(t)$ is calculated as:

$$
\begin{aligned}
net^{\langle c|p \rangle}_j(t) &= \sum_i^n \left[ \left( z_i y_{h}(t-1) + (1-z_i)\widetilde{y}_{i}(t)\right) v_{ji} + \theta_j\right]^{\langle c|p \rangle}\\
y^{\langle c|p \rangle}_j(t) &= f\left(net^{\langle c|p \rangle}_j(t)\right)\\
\end{aligned}
$$


$\Xi$  pronounced "ksee"

Dropout layers are used to prevent overfitting.

$\mathbf{\Xi}^{\langle c \rangle}(t)$ and $\mathbf{\Xi}^{\langle p \rangle}(t)$ are applied to $\mathbf{y}^{\langle c \rangle}(t)$ and $\mathbf{y}^{\langle p \rangle}(t)$, respectively. The output of the dropout layers is a replica of the input, with the exception that output from a fractional number of nodes, randomly selected with probability $\rho$ is pinned to zero. 

After applying dropout, the state of the model is as follows:


$$
\begin{aligned}
\Xi^{\langle c|p \rangle}_j(t) &= \xi \left(y^{\langle c|p \rangle}_j(t)\right)\\
\text{where } \xi(x) &= \begin{cases}
      0 & \text{ with probability } \rho\\
      x & \text{ otherwise}
   \end{cases}\\
\end{aligned}
$$

![Model Architecture](./report/model_architecture.png "Model Architecture")

A merge layer, $\mathbf{\Psi}^{\langle c,p \rangle}(t)$ is applied to the output of the two dropout layers. This layer is a simple vector concatenation, represented here by the $||$ operator.


$$
\begin{aligned}
\mathbf{\Psi}^{\langle c,p \rangle}(t) = \mathbf{\Xi}^{\langle c \rangle}(t) \; || \; \mathbf{\Xi}^{\langle p \rangle}(t)\\
\end{aligned}
$$

![Model Architecture](./report/model_architecture.png "Model Architecture")

The merged output feeds into a final GRU layer, $\mathbf{y}^{\langle m \rangle}(t)$ followed by two fully connected layers,  $\mathbf{y}^{\langle D1 \rangle}(t)$ and  $\mathbf{y}^{\langle D2 \rangle}(t)$ to produce the network output $\mathbf{y}^{\langle out \rangle}(t)$.

$$
\begin{aligned}
net^{\langle \Psi \rangle}_j(t) &= \sum_i^n \left[ \left( z_i y_{h}(t-1) + (1-z_i)\widetilde{y}_{i}(t)\right) w_{ji} + \theta_j\right]^{\langle \Psi \rangle}\\
y^{\langle \Psi \rangle}_j(t) &= f \left( net^{\langle \Psi \rangle}_j(t) \right) \\
\end{aligned}
$$

$$
\begin{aligned}
net^{\langle D1 \rangle}_j(t) &= \sum_j^m \left[y_j(t)w_{j} + \theta_j \right]^{\langle \Psi \rangle} \\
y^{\langle D1 \rangle}_j(t) &= \text{ReLU}\left( net^{\langle D1 \rangle}_j(t) \right) \\
\text{where ReLU}(x) &= \max(0,x)\\
\end{aligned}
$$

$$
\begin{aligned}
net^{\langle D2 \rangle}_j(t) &= \sum_j^m \left[y_j(t)w_{j} + \theta_j \right]^{\langle D1 \rangle} \\
y^{\langle D2 \rangle}_j(t) &= \text{softmax}\left( net^{\langle D2 \rangle}_j(t) \right) \\
\text{softmax}(x) &= e^x \left(\sum_{n}^m e^{x_n}\right)^{-1}\\
\end{aligned}
$$

$$
\begin{aligned}
y^{\langle out \rangle}_k(t) &= y^{\langle D2 \rangle}_j(t)\\
\end{aligned}
$$

![Model Architecture](./report/model_architecture.png "Model Architecture")

To keep calculation simple, we've been operating on individual neural units. As we've reached the output layer, it's important to remember that we're working with vectors:

$$
\begin{aligned}
\mathbf{y}^{\langle out \rangle}(t) &= \left[y^{\langle out \rangle}_0(t), ..., y^{\langle out \rangle}_k(t)\right] \\
\end{aligned}
$$

We now see why the softmax activation function is critical to the model -- the network's output always sums to one and is a valid representation of probability estimates for each character:


$$
\begin{aligned}
\sum_{i=0}^j \left[ y^{\langle out \rangle}_i(t), ..., y^{\langle out \rangle}_k(t) \right] &= 1\\
\end{aligned}
$$

"Differen-TA-tion"

### Backpropagation for a model of this complexity is not an easy task!

Fortunately, automatic differentiation frees us of this burden. This calculation is handled by Theano, the computation library wrapped by Keras.

# Training and Evaluation

### Training was performed on books obtained from Project Gutenberg. A single book was used for each model constructed. Models were trained for a minimum of 700 epochs.

### Amazon ```g2.2xlarge``` EC2 instances were used to perform training and evaluation. 

### Each epoch took approximately 230 seconds, equating to roughly 48 hours of computation per document.

###  We've provided a preconfigured AMI for those wishing to verify or expand upon our results without going through the trouble of resolving software dependencies. The AMI is publicly available as ```ami-2c3a7a4c```.

### To quantify the effect of part of speech information, models for *Pride and Prejudice* were trained with and without part of speech tags.

$$
\begin{aligned}
E\left[\left( \frac{\partial C}{\partial w} \right) ^2\right]^{\langle t \rangle}\!\!\!\!\!\!\! &= 0.9E\left[\left( \frac{\partial C}{\partial w} \right) ^2\right]^{\langle t-1 \rangle}\!\!\!\!\!\!\!\!\!\!\!\! + 0.1\left[\left( \frac{\partial C}{\partial w} \right) ^2\right]^{\langle t \rangle}\\
\theta^{\langle t + 1\rangle} &= \theta^{\langle t \rangle} - \frac{\eta}{\sqrt{E\left[\left( \frac{\partial C}{\partial w} \right) ^2\right]^{\langle t \rangle}} + \epsilon}\left[\frac{\partial C}{\partial w}\right]^{\langle t \rangle}\\
\end{aligned}
$$

# Results

### All models converged to a high level of accuracy within the training window. 

![Tags vs no tags](./report/images/tags_vs_no-tags.png "Tags vs no tags")

![Results](./report/images/results1.png "Results")

### Several interesting things to note
* The *Tale of Two Cities* model exhibited gradient instability around epoch 650
 * This is likely due to continued training after convergence.
 * GRUs improve gradient stability, but they're not perfect. 
 * This is a good example of the sometimes chaotic behavior of recurrent neural networks
* Our model effectively memorized *Alice in Wonderland*.
 * Overfitting should be avoided when producing generalized models, but it's not a huge issue here. 

# Concluding Thoughts

### Based on the accuracy of the document-specific models, we believe training an effective generalized model would be possible. 

### We did not attempt to train such a model due to the computational time required relative to the duration of this project. 

### SyntaxNet could further be utilized. Adding word dependency information would likely further improve model performance.

## Resources

### All of our code is available at 

```
https://github.com/davidcox143/rnn-text-compress
```

# Questions?